# Introduction to Neural Networks

Written by Rikab Gambhir (Center for Theoretical Physics, MIT)

In this tutorial, we will explore Neural Networks, the fundamental building block of deep learning. We will go into the very basics of the theory of Neural Networks and Universal Function Approximation. Then, we will explore practical immplementations of Neural Networks and deep learning that are widely used both in physics applications and also are widespread in industry.

This tutorial is divided into 4 parts:



1. **Neural Network Basics**: Constructing multi-layer perceptrons and studying universal function approximation.
2. **JAX**: An increasingly popular library used for machine learning. This library is extremeley similar to basic numpy, but has extra features like autodifferentiation and compilation that make it useful for machine learning.
3. **PyTorch**: A commonly used ML library. Developed by Meta. Especially nice for implementing fancy modern ML models, since they're mostly developed in PyTorch anyways!
4. **Tensorflow**: Less common in 2025, but many ML tools still use it.

**Prerequisites**

I will assume knowledge of the following:
1. Basic python and numpy. You should be comfortable with matrix operations within numpy, dealing with lists and loops, defining functions, and classes.
2. You are familiar with the previous tutorials on regression, classification, normalizing flows, and unsupervised learning. In particular you should appreciate the idea of finding parameters that minimize the log-likelhood (or other metrics) for function fitting, and the general importance of finding/optimizing for functions for statistical tasks.

# Chapter 1: Neural Network Basics

In [ ]:
# Standard Imports
import os
import sys
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In previous tutorials, e.g. [regression](https://colab.research.google.com/github/mcgen-ct/tutorials/blob/main/ml/regression.ipynb), the goal was to model a fixed functional form $f(x)$ where $f$ depended on some parameters $\theta$. For example, a linear fit of the form $f(x) = \theta_0 + \theta_1 x$.

In Deep Learning, we want to be more ambitious. We do not want to assume a specific functional form: rather than only ``searching'' over a fixed set of basis functions, we want to search over *all* functions, or at least a very large class of functions. Our strategy for doing this is to take a functional form with an extremeley large set of parameters, such that in the infinite parameter limit all functions of a particular class fit within the parameterization. For example. the set of functions:

\begin{align}
f(x) = \sum_{i=0}^N \theta_i x^i
\end{align}

models all one-dimension analytic functions as $N \to \infty$. However, we would like a more general parameterization that can work for many dimensions and even model non-smooth (or even non-continuous) functions arbtirarily well.

A **Neural Network (NN)**  (also known as a **Multilayer Perceptron (MLP)** a **feedforward network**, or a **Dense Neural Network (DNN)** depending on the context) parameterizes *all* peicewise-continuous functions from $\mathbb{R}^{n} \to \mathbb{R}^m$ arbitrarily well with a very simple parameterization.


To define a neural network, we first specify $L-2$ integers $N_1, ..., N_{L-1}$. Just for notation, choose $N_0 = n$ as the input dimension, and $N_L = m$ as the output dimension. $L$ is referred to as the *depth* of the network (or number of layers), and the $N$'s are the *width* of each layer. Unless you are doing something fancy (e.g. autoencoders), it is typical to choose $N$ to all be the same.

Then, we define a set of *layer functions*, which are maps $f^{\ell}:\mathbb{R}^{N_{\ell-1}}\to\mathbb{R}^{N_{\ell}}$, as:

$$ f^{\ell}(x) = \sigma(W^{(\ell)}x + b^{(\ell)})$$

where $W^{(\ell)} \in \mathbb{R}^{N_{\ell} \times N_{\ell -1}}$ and $b^{(\ell)} \in \mathbb{R}^{N_{\ell}}$ are the parameters that define the layer, and $\sigma$ is some pre-determined nonlinear transformation. This can differ between layers, but it is common to chose $\sigma$ to be the same for every layer except the last, where $\sigma$ is often instead chosen such that its image matches the desired output space. An extremeley common and simple chose for $\sigma$ is the ReLU (Rectified Linear Unit) function, which we will use throuhout the rest of this tutorial:

$$ \sigma(x) = \max(0, x)$$

Then, the full neural network is defined by:

$$ f = f^{L} \cdot f^{L-1} \cdot ... \cdot f^{1}$$

Let's make an MLP from scratch!

In [ ]:
# Building a Neural Network from Scratch #

input_dim = 2
output_dim = 1


L = 3
N = 16  # We will use the same N throughout for simplicity


# Function to initialize the W's and b's
# For now, lets just pick random numbers!
def init_params(input_dim, output_dim, L, N):
    Ws = []
    bs = []

    for l in range(L):
        if l == 0:
            W = np.random.randn(N, input_dim) / np.sqrt(input_dim)
            b = np.random.randn(N) / np.sqrt(input_dim)
            # The sqrt(input_dim) normalization is not important for our toy examples, but it is common to do for stability reasons

        elif l == L - 1:
            W = np.random.randn(output_dim, N) / np.sqrt(N)
            b = np.random.randn(output_dim) / np.sqrt(N)

        else:
            W = np.random.randn(N, N) / np.sqrt(N)
            b = np.random.randn(N) / np.sqrt(N)

        Ws.append(W)
        bs.append(b)

    return Ws, bs


# Implement the ReLU function
def sigma(x):
    return np.maximum(0, x)

In [ ]:
# Function to evaluate a neural network given x, the weights W, and the biases b


def MLP(x, Ws, bs):
    y = x.copy()

    for l in range(L):
        # Fun python fact: "@" implements matrix multiplication!
        y = Ws[l] @ y + bs[l]

        # Don't apply sigma to the final output so that our answer isn't forced positive
        if l != L - 1:
            y = sigma(y)

    return y

In [ ]:
# Test our MLP function by graphing the function f:R2 -> R1

# Define some test points in R2
xs1 = np.linspace(-1, 1, 100)
xs2 = np.linspace(-1, 1, 100)

xs1, xs2 = np.meshgrid(xs1, xs2)

# Initialize the weights and biases
Ws, bs = init_params(input_dim, output_dim, L, N)

ys = []
for x in zip(xs1.flatten(), xs2.flatten()):
    x = np.array(x)
    ys.append(MLP(x, Ws, bs))

ys = np.array(ys)
ys = ys.reshape(xs1.shape)

# 3d plot
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
ax.plot_surface(xs1, xs2, ys)
ax.set_xlabel("x1")
ax.set_ylabel("x2")
ax.set_zlabel("y = MLP(x)")
plt.show()

**A note on functional vs. object-oriented programming**

In the above code, we defined our MLP purely using python functions. There is no neural network "object" with an internal state keeping track of the parameters. Instead, the parameters $W$ and $b$ are also treated as inputs to functions. This is *functional programming*, in which there are no objects with internal states that get modified. This is the approach to ML used by JAX.

It is also possible to define an MLP *class*, which is an object that contains the parameters as internal states that can potentially be modified, and methods that implement the model and evaluate $f(x)$. This is the approach to ML used by PyTorch and Tensorflow.

It is largely a matter of programming taste which you prefer. Below, we will see a brief example of the above code, but written in an object-oriented style rather than functional.

In [ ]:
class My_MLP_Class:
    def __init__(self, input_dim, output_dim, L, N):
        # Initialize the network arguments
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.L = L
        self.N = N

        # Initialize the network internal state using the same initi function
        self.Ws, self.bs = init_params(input_dim, output_dim, L, N)

    def evaluate(self, x):
        # Just use the same exact function as above
        return MLP(x, self.Ws, self.bs)

    # "Magic Method" that lets us call the class as if it were a function (just syntatic magic)
    def __call__(self, x):
        return self.evaluate(x)


my_MLP = My_MLP_Class(input_dim, output_dim, L, N)

# Access the weights
my_weights = my_MLP.Ws
print("The number of layers is ", len(my_weights), ",Expected 3")

# Evaluate the function
print("f(1,1) = ", my_MLP(np.array([1, 1])))

**Historical Notes and Semantics**

The case where $L = 2$ (no ``hidden layers'' between the input and output) with the output dimensionality is $1$ is called a perceptron historically. These were introduced with $\sigma$ not as ReLU, but rather:
$$\sigma(x) = \frac{1}{1 + e^{-x}}$$
(the sigmoid function, hence the notation), and were used back in the day as a model of a biological neuron. The neuron "activates" (produces $1$) when $x$ is large, and "deactivates" (produces $0$ when $x$ is small, where $b$ is then a bias. For this reason, $\sigma$ is called an activation function. This is also why our models are called "Neural Networks". The "network" is because the parameters of the weight matrix $w_{ij}$ are drawn as lines connecting a node $i$ in the previous layer to a node $j$ in the next layer. It's important to remember though, that these are just affine transformations interleaved by some simple nonlinear functions, and there isn't really anything magic here, just slightly-nonlinear algebra.

The name "feedforward" network just refers to the function-compositional aspect of the model. It is to be contrasted with a "backwards pass", where derivatives with respect to the network are actually computed in reverse-order due to chain-rule simplifications. The name "dense" neural network is to emphasize that this is the simplest possible network one can build. There are many modern models with additional properties (such as gauranteeing symmetries, or working in spaces other than simple vector spaces, or deliberately constraining the function space), but many of these can be reduced to very large MLPs with constrained weights. When we say "dense" or "fully-conencted" MLPs, we typically mean there are no constraints on the parameters.

## Chapter 1.1: Universal Function Approximation

The power of MLPs is that they are an efficient way to parameterize a large class of functions. This is captured by the **Universal Function Approximation Theorem(s) (UFAT)** (there are lots of variants, but at the level of rigor we are working at, we won't worry about this).

**Emotionally**, the UFAT tells us that for sufficiently large $N$ and $L$, an MLP can approximate any (reasonable) $n$-to-$m$ dimensional function arbitrarily well.

**Slightly more precisely**, a version of UFAT says: For any piecewise-continuous function $f: \mathbb{R}^n \to \mathbb{R}^m$ defined on a compact domain $D \subset \mathbb{R}^n$, and for any "error tolerance" $\epsilon  > 0$, there exists a large enough $N$ and $L$ such that one can define an MLP with specially-chosen parameters $W$ and $b$ such that:
$$ \int_D dx |f(x) - MLP(x)| < \epsilon $$
i.e. that we have approximated the function to within the specified error.

[Side note: It is actually always possible to do this with just $L$ = 3 (meaning just one hidden layer with chosen $N$ in our defined $L$ counting), but typically this requires an exponentially large $N$ and isn't of practical use for what we will be doing].


We will not prove the UFAT. However, we will explore a weaker-version of it that is easier to understand: If instead we explore continuous-and-piecewise-once-differentiable functions rather than just piecewise-continuous, then there is an easy construction using ReLU networks. If a function is piecewise-once-differentiable, then it can be well-approximated by a piecewise-linear function. We will see below (as exercises) how ReLU networks can exactly reproduce piecewise linear functions.

#### Exercise: Modeling |x|

Given $f(x) = |x|$ in 1 dimension, design an MLP with a choice of N, L, weights W, and biases b that *exactly* match $f(x)$.

HINT: It is possible to do this with $L = 2$ (one hidden layer) and $N = 2$.

HINT 2: It is possible to do this with $b = 0$.

In [ ]:
def f(x):
    return np.abs(x)


L = 0  # YOUR SOLUTION HERE
N = 0  # YOUR SOLUTION HERE

Ws = []  # YOUR SOLUTION HERE
bs = []  # YOUR SOLUTION HERE

xs = np.linspace(-1, 1, 100)

# Evaluate the solution
ys = []
for x in xs:
    x = np.array([x])
    ys.append(MLP(x, Ws, bs))

ys = np.array(ys)

# Plot
plt.plot(xs, f(xs), label="f(x)")
plt.plot(xs, ys, label="MLP(x)")
plt.legend()
plt.show()

### Exercise: Approximating a smooth 1D function.

Given $f(x) = \sin(10x) \exp(-2x^2)$ on the interval $[-1,1]$, design an MLP with ReLU-activations that approximates the function to within an error of $\epsilon < 0.01$ (where error is the mean-absolute error, as defined above). As a bonus, your implementation should be systematically improvable, e.g. it should be straightforward to make the MLP bigger to reduce the error further. Don't cheat and use minimization to get the parameters, explicitly construct them!


HINT: First construct a continuous piecewise linear appoximation to the function, then implement this piecewise linear function as an MLP. It is possible to do this without knowledge of the actual form of $f$.

HINT 2: This is possible to do systematically with $L = 2$ as before, but with a very large $N$. My personal solution requires $N$ between 100 and 150.

HINT 3: A piecewise-linear continuous function can be written as $f(x) = c_0 + m_0x + + \sum_{j = 1}^{n-1}(m_j - m_{j-1})\sigma(x - x_j)$, where $\sigma$ is ReLU, $x_{1}...x_{n-1}$ are the internal breakpoints, $m_j$ are the slopes to the right of each breakpoint, and $c_0$ is the $y$-coordinate at the leftmost point.

In [ ]:
def f(x):
    return np.sin(10 * x) * np.exp(-2 * x**2)


L = 0  # YOUR SOLUTION HERE
N = 0  # YOUR SOLUTION HERE
Ws = (
    []
)  # YOUR SOLUTION HERE (hint, define a function to construct Ws, bs, systematically)
bs = []  # YOUR SOLUTION HERE


xs = np.linspace(-1, 1, 1000)

ys = []
for x in xs:
    x = np.array([x])
    ys.append(MLP(x, Ws, bs))
ys = np.array(ys)

plt.plot(xs, f(xs), label="f(x)")
plt.plot(xs, ys, label="MLP(x)")
plt.legend()
plt.show()

# Approximate the mean absolute error
print(
    "Mean absolute error: ", np.mean(np.abs(ys[:, 0] - f(xs)))
)  # [:,0] due to annoying indexing